In [1]:
import requests
import pandas as pd
from datetime import datetime
import json

# Your API key
API_KEY = "f2IedVpstXz7qOfWiHl86s8BzVdQBMSC"

def get_spy_historical_data(api_key, from_date=None, to_date=None):
    """
    Download historical SPY data from FMP
    
    Parameters:
    - api_key: Your FMP API key
    - from_date: Start date (YYYY-MM-DD format)
    - to_date: End date (YYYY-MM-DD format)
    """
    base_url = "https://financialmodelingprep.com/api/v3/historical-price-full/SPY"
    
    params = {
        "apikey": api_key
    }
    
    if from_date:
        params["from"] = from_date
    if to_date:
        params["to"] = to_date
    
    try:
        response = requests.get(base_url, params=params)
        response.raise_for_status()
        
        data = response.json()
        
        if "historical" in data:
            df = pd.DataFrame(data["historical"])
            df['date'] = pd.to_datetime(df['date'])
            df = df.sort_values('date')
            return df
        else:
            print("No historical data found")
            return None
            
    except requests.exceptions.RequestException as e:
        print(f"Error fetching data: {e}")
        return None

def get_spy_quote(api_key):
    """
    Get current SPY quote data from FMP
    """
    base_url = "https://financialmodelingprep.com/api/v3/quote/SPY"
    
    params = {
        "apikey": api_key
    }
    
    try:
        response = requests.get(base_url, params=params)
        response.raise_for_status()
        
        data = response.json()
        return data[0] if data else None
        
    except requests.exceptions.RequestException as e:
        print(f"Error fetching quote: {e}")
        return None

def get_spy_intraday_data(api_key, interval="5min"):
    """
    Get intraday SPY data
    
    Parameters:
    - interval: '1min', '5min', '15min', '30min', '1hour'
    """
    base_url = f"https://financialmodelingprep.com/api/v3/historical-chart/{interval}/SPY"
    
    params = {
        "apikey": api_key
    }
    
    try:
        response = requests.get(base_url, params=params)
        response.raise_for_status()
        
        data = response.json()
        
        if data:
            df = pd.DataFrame(data)
            df['date'] = pd.to_datetime(df['date'])
            df = df.sort_values('date')
            return df
        else:
            print("No intraday data found")
            return None
            
    except requests.exceptions.RequestException as e:
        print(f"Error fetching intraday data: {e}")
        return None

def save_data_to_csv(df, filename):
    """Save DataFrame to CSV file"""
    if df is not None:
        df.to_csv(filename, index=False)
        print(f"Data saved to {filename}")
    else:
        print("No data to save")

# Example usage
if __name__ == "__main__":
    # Get historical daily data
    print("Fetching historical SPY data...")
    historical_data = get_spy_historical_data(
        API_KEY,
        from_date="2024-01-01",
        to_date="2024-12-31"
    )
    
    if historical_data is not None:
        print(f"Downloaded {len(historical_data)} days of data")
        print("\nFirst 5 rows:")
        print(historical_data.head())
        
        # Save to CSV
        save_data_to_csv(historical_data, "spy_historical_2024.csv")
    
    # Get current quote
    print("\n\nFetching current SPY quote...")
    quote = get_spy_quote(API_KEY)
    if quote:
        print(f"Current Price: ${quote.get('price', 'N/A')}")
        print(f"Day Change: {quote.get('changesPercentage', 'N/A')}%")
        print(f"Volume: {quote.get('volume', 'N/A'):,}")
    
    # Get intraday data
    print("\n\nFetching intraday SPY data...")
    intraday_data = get_spy_intraday_data(API_KEY, interval="5min")
    
    if intraday_data is not None:
        print(f"Downloaded {len(intraday_data)} intraday data points")
        print("\nLatest 5 data points:")
        print(intraday_data.tail())
        
        # Save to CSV
        save_data_to_csv(intraday_data, "spy_intraday_5min.csv")

Fetching historical SPY data...
Downloaded 252 days of data

First 5 rows:
          date    open    high     low   close  adjClose     volume  \
251 2024-01-02  472.16  473.67  470.49  472.65    463.89  123623700   
250 2024-01-03  470.43  471.19  468.17  468.79    460.11  103585900   
249 2024-01-04  468.30  470.96  467.05  467.28    458.62   84232200   
248 2024-01-05  467.49  470.44  466.43  467.92    459.25   86118913   
247 2024-01-08  468.43  474.75  468.30  474.60    465.81   74879100   

     unadjustedVolume  change  changePercent      vwap           label  \
251         123623700    0.49       0.103780  472.2425  January 02, 24   
250         103585900   -1.64      -0.348620  469.6450  January 03, 24   
249          84232200   -1.02      -0.217810  468.3975  January 04, 24   
248          86118913    0.43       0.091981  468.0700  January 05, 24   
247          74879100    6.17       1.320000  471.5200  January 08, 24   

     changeOverTime  
251        0.001038  
250      

In [ ]:
intraday_data to use 